In [2]:
import pandas as pd
import numpy as np
#import json # library to handle JSON files
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup
import requests


In [6]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')

#print(soup.prettify())

 1- Extracting the 3 tables from the webpage and cleaning up Not Assigned
 

In [7]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

#print(table)
# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

2 - Consolidation on unique postal codes


In [8]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 84


3 - Moving Data into Pandas Dataframe

In [13]:
toronto_dict = {'Postal Code':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Postal Code,Borough,Neighbourhood
0,M3H,North York,"Bathurst Manor, Wilson Heights"
1,M9P,Etobicoke,Westmount
2,M8X,Etobicoke,The Kingsway
3,M3A,North York,Parkwoods
4,M3C,North York,Flemingdon Park
5,M4E,East Toronto,The Beaches
6,M5B,Downtown Toronto,Ryerson
7,M1H,Scarborough,Cedarbrae
8,M3K,North York,CFB Toronto
9,M6M,York,"Keelsdale, Mount Dennis, Silverthorn"


In [10]:

df_toronto.shape

(84, 3)

4- Merging the Coordinates with the Postal Code (using the csv as Geocoder failed)

In [11]:
df_geo = pd.read_csv('Downloads/Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df_final = pd.merge(df_toronto, df_geo, on='Postal Code')
df_final.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3H,North York,"Bathurst Manor, Wilson Heights",43.754328,-79.442259
1,M9P,Etobicoke,Westmount,43.696319,-79.532242
2,M8X,Etobicoke,The Kingsway,43.653654,-79.506944
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M3C,North York,Flemingdon Park,43.725900,-79.340923
5,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
7,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
8,M3K,North York,CFB Toronto,43.737473,-79.464763
9,M6M,York,"Keelsdale, Mount Dennis, Silverthorn",43.691116,-79.476013
